In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'Yu Mincho'
from IPython.core.display import display
%matplotlib inline

# データの保存場所を指定。
# この場合は、C:/sample/linear_regression/を読み書きする。
WORK_DIR = 'C:/sample/linear_regression/'
DATA_DIR = 'C:/sample/linear_regression/data/'

# 2.2.1で作ったデータを読み込む
data_for_analysis_loaded = pd.read_pickle(
    f'{DATA_DIR}data_for_analysis_fin.pickle'
)

In [ ]:
independent_variables_names = [
    'market_beta', '企業規模', '簿価時価比率', '財務レバレッジ',
    '赤字ダミー', '25日移動平均乖離率', 'PER'
]
columns_to_use = [
    '業種', '翌日収益率', '翌日超過収益率', '収益率', '市場収益率'
] + independent_variables_names

data_for_analysis=data_for_analysis_loaded.assign(
    # 超過収益率を1日分ずらして、Y_iを作成
    翌日超過収益率=lambda x: x['超過収益率'].groupby(level=0).shift(-1),
    # 後のシミュレーションで利用するので一緒に作成しておく
    翌日収益率=lambda x: x['収益率'].groupby(level=0).shift(-1)  
)[columns_to_use]

In [ ]:
# 証券コードごとに5つずつ表示して、計算結果を確認
data_for_analysis.dropna(
    subset=independent_variables_names
).groupby(level=0).head()

In [ ]:
exclude_fin = data_for_analysis[
    data_for_analysis['業種'].apply(
        lambda x: x not in ['銀行', '証券・先物', '保険', 'その他金融']
    )
]

In [ ]:
data_tmp = exclude_fin.dropna(
    subset=independent_variables_names
).xs(pd.datetime(2016,7,6), level=1)  # 2016年7月6日のデータを取得
exog = sm.add_constant(data_tmp[independent_variables_names])
endog = data_tmp['翌日超過収益率']
sm.OLS(endog, exog).fit().summary()

In [ ]:
data_tmp = exclude_fin.dropna(
    subset=independent_variables_names
).xs(pd.datetime(2017,7,6), level=1)  
exog = sm.add_constant(data_tmp[independent_variables_names])
endog = data_tmp['翌日超過収益率']
sm.OLS(endog, exog).fit().summary()

In [ ]:
def cross_sectional_regression_overtime(
    data_with_excess_returns,
    endog_name, exog_names
):
    group_by_date = data_with_excess_returns.groupby('日時')
    
    results = []
    for date_point, values in tqdm(group_by_date):
        
        result = cross_sectional_regression(
            values,
            endog_name,
            exog_names
        )
        if result is None:
            continue
        results.append(result)
    
    results = pd.concat(results)
    return results

In [ ]:
def cross_sectional_regression(data, endog_name, exog_names):
    data = data.reset_index()
    data = data.dropna(subset=endog_name + exog_names)

    if data.shape[0] < 1:  # 空のDataFrameは無視する
        return None

    end_date = data['日時'].max()

    endog = data[endog_name]

    exog = data[exog_names]
    exog = exog.assign(constant=1)

    model = sm.OLS(endog, exog)

    result = model.fit()
    betas = result.params.rename(end_date)

    result = pd.DataFrame(betas).T

    return result

In [ ]:
def calculate_mean_value_of_coefficients(coefficients):
    mean = coefficients.mean().rename('mean')
    std_err = (
        coefficients.std() / np.sqrt(coefficients.shape[0])
    ).rename('std err')
    t_stat = (mean / std_err).rename('t-stat')
    
    result = pd.concat([mean, std_err, t_stat], axis=1)
    
    return result

In [ ]:
coefficients_excluding_fin = cross_sectional_regression_overtime(
    exclude_fin, endog_name=['翌日超過収益率'],
    exog_names=independent_variables_names
)

# 結果を出力して確認
display(coefficients_excluding_fin.head())  

In [ ]:
result = calculate_mean_value_of_coefficients(
    coefficients_excluding_fin
)

# 結果を出力して確認
display(result)

In [ ]:
result[['mean']].applymap(lambda x: '{:.2%}'.format(x * 250))  # 表2-1を作成

In [ ]:
window_size = 50
rolling_coefficients = coefficients_excluding_fin.rolling(
    window_size
).mean()

rolling_std_err = (
    coefficients_excluding_fin.rolling(window_size).std()
    / np.sqrt(window_size)
)

rolling_coefficients.index.rename('日付', inplace=True)

fig, axes=plt.subplots(4, 2, figsize=(12, 12))

for col, ax in zip(rolling_coefficients.columns, axes.flatten()):
    sns.lineplot(data=rolling_coefficients[col], ax=ax)
    sns.lineplot(
        data=rolling_coefficients[col] + 2 * rolling_std_err[col],
        ax=ax,
        color='gray'
    )
    sns.lineplot(
        data=rolling_coefficients[col] - 2 * rolling_std_err[col],
        ax=ax,
        color='gray'
    )
    ax.hlines( # y = 0の破線を引く
        0, *ax.get_xlim(), 
        linestyles=':', alpha=.5
    )    
    for line in ax.get_lines()[1:]:
        line.set_linestyle('--')
    ax.set_title(col if col != 'PER' else 'E/P ratio')

fig.tight_layout() # グラフ同士の間隔を広げる

In [ ]:
def create_portfolio_by_one_variable(
    data,
    sort_by,
    q,
    labels=None,
    group_name=None
):
    group_by_date = data.groupby('日時')
    
    if isinstance(q, int) and labels is None:
        labels = range(q)

    values = []
    for date, value in group_by_date:
        if value[sort_by].isnull().all(): # 空のDataFrameは無視する
            continue
            
        value = value.assign(
            quantile=lambda x: pd.qcut(
                x[sort_by], q, labels=labels
            )
        )
        
        if group_name is not None:
            value.rename(columns={'quantile':group_name}, inplace=True)
            
        values.append(value)
    
    return pd.concat(values)

In [ ]:
portfolio_by_mv = create_portfolio_by_one_variable(
    data=exclude_fin,
    sort_by='25日移動平均乖離率',
    q=5,
    labels=range(5),
    group_name='MV_quantile'
)

In [ ]:
portfolio_by_mv.head()

In [ ]:
portfolio_returns = portfolio_by_mv.groupby(
    ['MV_quantile', '日時']
)['翌日収益率'].mean()

In [ ]:
market_return = exclude_fin.groupby(
    '日時'
)['翌日収益率'].mean().rename('単純平均リターン')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

to_plot=portfolio_returns.unstack().T.filter(
    items=[0, 4]
).rename(
    columns={0:'乖離率 低', 4:'乖離率 高'}
).join(
    market_return
).dropna(
    how='all'
).apply(
    lambda column: np.log((column + 1).cumprod())
)
to_plot.columns.rename('ポートフォリオ', inplace=True)

sns.lineplot(
    data=to_plot.stack().reset_index(name='累積リターン'),
    x='日時',
    y='累積リターン',
    hue='ポートフォリオ',
    style='ポートフォリオ',
    palette='gray',
    ax=ax
)

In [ ]:
# ポートフォリオの構築
portfolio_by_per = create_portfolio_by_one_variable(
    data=exclude_fin,
    sort_by='PER',
    q=5,
    labels=range(5),
    group_name='PER_quantile'
)

# 累積リターンの計算
portfolio_returns = portfolio_by_per.groupby(
    ['PER_quantile', '日時']
)['翌日収益率'].mean()

to_plot=portfolio_returns.unstack().T.filter(
    items=[0, 4]
).rename(
    columns={0:'低E/P ratio', 4:'高E/P ratio'}
).join(
    market_return
).dropna(
    how='all'
).apply(
    lambda column: np.log((column + 1).cumprod())
)
to_plot.columns.rename('ポートフォリオ', inplace=True)
_, ax=plt.subplots(figsize=(12, 5))
sns.lineplot(
    data=to_plot.stack().reset_index(name='累積リターン'),
    x='日時',
    y='累積リターン',
    hue='ポートフォリオ',
    style='ポートフォリオ',
    palette='gray',
    ax=ax
)

In [ ]:
# プロットして比較
fig, ax = plt.subplots(figsize=(12, 5))
handles, labels = [], []
hml = portfolio_returns.loc[4].rolling(25).sum() - portfolio_returns.loc[0].rolling(25).sum()
ax=hml.rename('割安・成長のパフォーマンス差').plot(color='k', ax=ax)
handle, label= ax.get_legend_handles_labels()
handles.extend(handle)
labels.extend(label)

ax=(np.log(market_return.apply(lambda x:x**2).rolling(10).sum()[hml.index])).plot(
    ax=ax, secondary_y=True, linestyle=':', color='gray'
)
handle, label= ax.get_legend_handles_labels()
handles.extend(handle)
labels.extend(label)

ax.set_title('割安・成長ポートフォリオのパフォーマンス差とボラティリティ')
ax.legend(handles, labels, bbox_to_anchor=(.5,-.2), ncol=2, loc='upper center')

In [ ]:
def calculate_predicted_values(
    exog,
    shifted_coefficients,
    predicted_value_label='predicted_value'
):
    predicted_returns=[]
    
    group_by_date = exog.groupby(level=1)
    for date, group in tqdm(group_by_date):
        try:
            coefficients_ = shifted_coefficients.xs(date).T

            predicted = np.dot(group.values, coefficients_)
            predicted_returns.append(
                pd.DataFrame(
                    predicted,
                    columns=[predicted_value_label],
                    index=group.index
                )
            )
        except KeyError:
            continue

    predicted_returns = pd.concat(predicted_returns).sort_index()

    return predicted_returns

In [ ]:
# ファクターと実測値の準備
X = exclude_fin.assign(constant=1)[coefficients_excluding_fin.columns]

In [ ]:
predicted = calculate_predicted_values(
    X,
    coefficients_excluding_fin.shift(),
    '予測リターン'
)

data_with_predicted_values = predicted.join(exclude_fin)

data_with_predicted_values.head()

In [ ]:
portfolio_by_predicted = create_portfolio_by_one_variable(
    data=data_with_predicted_values,
    sort_by='予測リターン',
    q=5,
    labels=range(5),
    group_name='predicted_quantile'
)

In [ ]:
# 累積リターンの計算
portfolio_returns = portfolio_by_predicted.groupby(
    ['predicted_quantile', '日時']
)['翌日収益率'].mean()

# プロットして比較
fig, ax = plt.subplots(figsize=(12, 5))

to_plot=portfolio_returns.unstack().T.filter(
    items=[0, 4]
).rename(
    columns={0:'予測値 低', 4:'予測値 高'}
).join(
    market_return
).dropna(
    how='all'
).apply(
    lambda column: np.log((column + 1).cumprod())
)
to_plot.columns.rename('ポートフォリオ', inplace=True)
sns.lineplot(
    data=to_plot.stack().reset_index(name='累積リターン'),
    x='日時',
    y='累積リターン',
    hue='ポートフォリオ',
    style='ポートフォリオ',
    palette='gray',
    ax=ax
)


ax.set_title('予測値に基づくポートフォリオの累積リターン')
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

pos_ = portfolio_by_predicted[
    (portfolio_by_predicted['予測リターン'] > 0)
].groupby('日時')['翌日収益率'].mean() + 1

neg_ = portfolio_by_predicted[
    (portfolio_by_predicted['予測リターン']<=0)
].groupby('日時')['翌日収益率'].mean()+1

to_plot = pd.concat(
    [
        np.log(pos_).cumsum().rename('予測値 正'),
        np.log(neg_).cumsum().rename('予測値 負'),
        np.log(market_return[neg_.index] + 1).cumsum()
    ],
    axis=1
)
to_plot.columns.rename('ポートフォリオ', inplace=True)
sns.lineplot(
    data=to_plot.stack().reset_index(name='累積リターン'),
    x='日時',
    y='累積リターン',
    hue='ポートフォリオ',
    style='ポートフォリオ',
    palette='gray',
    ax=ax
)

In [ ]:
test_predicted = cross_sectional_regression_overtime(
    data_with_predicted_values,
    endog_name=['翌日超過収益率'],
    exog_names=['予測リターン']
)

calculate_mean_value_of_coefficients(test_predicted)

In [ ]:
data_for_analysis_loaded